In [1]:
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/MyDrive/[2024-2025]AN2DL/Homework1


Mounted at /gdrive
/gdrive/MyDrive/[2024-2025]AN2DL/Homework1


In [2]:
%%capture
!pip install keras_cv
import keras_cv

In [3]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # @param ["tensorflow", "jax", "torch"]

import matplotlib.pyplot as plt


# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
import keras as tfk
from keras.layers import Input, Dense, Dropout, Lambda, BatchNormalization, Conv2D, MaxPooling2D, Flatten
from keras import layers as tfkl
import keras_cv

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(f"Tensorflow version -> {tf.__version__}")
print(f"Keras version -> {tfk.__version__}")

# Import other libraries
import requests
from io import BytesIO
import cv2
from PIL import Image
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
import seaborn as sns


# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

Tensorflow version -> 2.17.1
Keras version -> 3.5.0


In [4]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
tfds.disable_progress_bar()

import numpy as np


data = np.load(
        'cleaned_data_dedup.npz'
)
X = data['images']
y = data['labels']



In [5]:
class_mapping = {
    0: "Basophil",
    1: "Eosinophil",
    2: "Erythroblast",
    3: "Immature granulocytes",
    4: "Lymphocyte",
    5: "Monocyte",
    6: "Neutrophil",
    7: "Platelet"
}

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed, stratify=y_temp)

class_names =  list(class_mapping.values())

#y_train = tfk.utils.to_categorical(y_train, num_classes=len(class_names))
#y_val = tfk.utils.to_categorical(y_val, num_classes=len(class_names))

data_train = tf.data.Dataset.from_tensor_slices((X_train, y_train.flatten())).batch(BATCH_SIZE, drop_remainder=True)
data_val = tf.data.Dataset.from_tensor_slices((X_val, y_val.flatten())).batch(BATCH_SIZE, drop_remainder=True)
data_test = tf.data.Dataset.from_tensor_slices((X_test, y_test.flatten())).batch(BATCH_SIZE, drop_remainder=True)

print(data_val)
#train_steps_per_epoch = X.shape[0] // BATCH_SIZE
#val_steps_per_epoch = X.shape[0] // BATCH_SIZE

<_BatchDataset element_spec=(TensorSpec(shape=(32, 96, 96, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(32,), dtype=tf.uint8, name=None))>


In [6]:
IMAGE_SIZE = (96, 96)
num_classes = 8


def to_dict(image, label):
    #image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32)
    label = tf.one_hot(label, num_classes)

    return {"images": image, "labels": label}


def prepare_dataset(dataset, split):
    if split == "train":
        return (
            dataset.shuffle(10 * BATCH_SIZE)
            .map(to_dict, num_parallel_calls=AUTOTUNE)
        )
    else:
        return dataset.map(to_dict, num_parallel_calls=AUTOTUNE)


def load_dataset(split="train"):
    if split == "train":
      dataset = data_train
    elif split == "val":
      dataset = data_val

    else:
      dataset = data_test
    return prepare_dataset(dataset, split)


train_dataset = load_dataset()

## Training a CNN

In [7]:

def preprocess_for_model(inputs):
    images, labels = inputs["images"], inputs["labels"]
    images = tf.cast(images, tf.float32)
    return images, labels

IMAGE_SIZE = (96, 96)
num_classes = 8


def to_dict(image, label):
    #image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32)
    label = tf.one_hot(label, num_classes)

    return {"images": image, "labels": label}


def prepare_dataset(dataset, split):
    if split == "train":
        return (
            dataset.shuffle(10 * BATCH_SIZE)
            .map(to_dict, num_parallel_calls=AUTOTUNE)
        )
    else:
        return dataset.map(to_dict, num_parallel_calls=AUTOTUNE)


def load_dataset(split="train"):
    if split == "train":
      dataset = data_train
    elif split == "val":
      dataset = data_val

    else:
      dataset = data_test
    return prepare_dataset(dataset, split)


train_dataset = load_dataset()

rand_augment = keras_cv.layers.RandAugment(
    value_range=(0, 255),
    augmentations_per_image=3,
    magnitude=0.3,
    magnitude_stddev=0.2,
    rate=1.0,
)

cut_mix = keras_cv.layers.CutMix()
mix_up = keras_cv.layers.MixUp()


def cut_mix_and_mix_up(samples):
    samples = cut_mix(samples, training=True)
    samples = mix_up(samples, training=True)
    return samples

def apply_rand_augment(inputs):
    inputs["images"] = rand_augment(inputs["images"])
    return inputs


train_dataset = load_dataset().map(apply_rand_augment, num_parallel_calls=AUTOTUNE)

train_dataset = (
    load_dataset()
    .map(apply_rand_augment, num_parallel_calls=AUTOTUNE)
    .map(cut_mix_and_mix_up, num_parallel_calls=AUTOTUNE)
)

train_dataset = train_dataset.map(preprocess_for_model, num_parallel_calls=AUTOTUNE)

val_dataset = load_dataset(split="val")
val_dataset = val_dataset.map(preprocess_for_model, num_parallel_calls=AUTOTUNE)

test_dataset = load_dataset(split="test")
test_dataset = test_dataset.map(preprocess_for_model, num_parallel_calls=AUTOTUNE)

train_dataset = train_dataset.prefetch(AUTOTUNE)
test_dataset = test_dataset.prefetch(AUTOTUNE)


In [18]:
def fine_tune_focal():
    ft_model = tfk.models.load_model('Fine_Fine_tuneEfficientNet_98.78.keras')

    ft_model.get_layer('efficientnetb7').trainable = True

    # Set all MobileNetV3Small layers as non-trainable
    for layer in ft_model.get_layer('efficientnetb7').layers:
        layer.trainable = False

    # Enable training only for Conv2D and DepthwiseConv2D layers
    for i, layer in enumerate(ft_model.get_layer('efficientnetb7').layers):
        if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
            layer.trainable = True
            #print(i, layer.name, type(layer).__name__, layer.trainable)

    N_freeze = 260

    for layer in ft_model.get_layer('efficientnetb7').layers[:N_freeze]:
        layer.trainable = False
        # print(layer.name, layer.trainable)

    ft_model.compile(
        loss=tfk.losses.CategoricalFocalCrossentropy(
            alpha=[0.1,0.1,0.1,0.145,0.1,0.1,0.1,0.1],
            gamma=5.0,
            from_logits=False,
            label_smoothing=0.1,
            axis=-1,
            reduction='sum_over_batch_size',
            name='categorical_focal_crossentropy'
        ),
        optimizer=tfk.optimizers.RMSprop(learning_rate = 1e-4),
        metrics=['accuracy']
    )

    return ft_model


In [ ]:
callbacks = [
    tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
    tfk.callbacks.ReduceLROnPlateau(monitor='FocalCELoss_', patience=4, factor=0.5, min_lr=1e-6)
]

model = fine_tune_focal()
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=val_dataset,
    callbacks=callbacks,

).history

final_val_accuracy = round(max(history['val_accuracy']) * 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

file_name = "FocalCELoss_" +  model.name + "_" + str(final_val_accuracy) + '.keras'

model.save(file_name)

Epoch 1/50
290/290 ━━━━━━━━━━━━━━━━━━━━ 118s 115ms/step - accuracy: 0.7375 - loss: 0.0716 - val_accuracy: 0.9696 - val_loss: 0.0306 - learning_rate: 1.0000e-04
Epoch 2/50
290/290 ━━━━━━━━━━━━━━━━━━━━ 21s 65ms/step - accuracy: 0.7112 - loss: 0.0601 - val_accuracy: 0.9852 - val_loss: 0.0253 - learning_rate: 1.0000e-04
Epoch 3/50
290/290 ━━━━━━━━━━━━━━━━━━━━ 21s 66ms/step - accuracy: 0.7206 - loss: 0.0572 - val_accuracy: 0.9826 - val_loss: 0.0219 - learning_rate: 1.0000e-04
Epoch 4/50
290/290 ━━━━━━━━━━━━━━━━━━━━ 21s 65ms/step - accuracy: 0.7401 - loss: 0.0563 - val_accuracy: 0.9844 - val_loss: 0.0210 - learning_rate: 1.0000e-04
Epoch 5/50
290/290 ━━━━━━━━━━━━━━━━━━━━ 21s 65ms/step - accuracy: 0.7389 - loss: 0.0561 - val_accuracy: 0.9800 - val_loss: 0.0209 - learning_rate: 1.0000e-04
Epoch 6/50
290/290 ━━━━━━━━━━━━━━━━━━━━ 21s 65ms/step - accuracy: 0.7432 - loss: 0.0553 - val_accuracy: 0.9835 - val_loss: 0.0205 - learning_rate: 1.0000e-04
Epoch 7/50
290/290 ━━━━━━━━━━━━━━━━━━━━ 21s 65ms/s

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_dataset)